# 1. Importar librerías

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

from importlib import reload
import utils.utils as utils
reload(utils)  # Recarga el módulo

<module 'utils.utils' from 'c:\\Users\\USUARIO\\OneDrive - Universidad de Antioquia\\Aprendizaje\\Universidad\\2023-2\\Analítica II\\Caso Estudio No Supervisado\\no_supervisado\\utils\\utils.py'>

# 2. Importar base de datos

In [2]:
df = pd.read_csv('datasets/data_limpio.csv')

In [3]:
df.head()

,tipo_entidad,fecha_corte,unicap,desc_unicap,renglon,desc_renglon,saldo_a_la_fecha,vigente,v1_2,v2_3,...,riesgo_a_c,riesgo_a_s,riesgo_b_c,riesgo_b_s,riesgo_c_c,riesgo_c_s,riesgo_d_c,riesgo_d_s,riesgo_e_c,riesgo_e_s
0,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,37651.35,34981.44,768.06,579.92,...,6176,32878.87,270,1835.21,142,975.34,326,1368.70,69,593.09
1,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,826852.23,782016.60,9271.46,8065.65,...,125128,772666.40,1840,11740.70,1486,9872.85,1854,14111.82,2469,18460.45
2,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,108451.56,104345.67,1359.38,845.85,...,9723,100426.47,324,2920.77,207,1936.82,165,1623.50,144,1544.01
3,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,209307.78,196907.74,6328.34,2236.77,...,91873,196131.43,4802,6541.01,2385,2574.77,2515,3677.98,160,382.60
4,ESTABLECIMIENTOS BANCARIOS,2015-01-31,1,CRÉDITO ROTATIVO,5,CRÉDITO ROTATIVO TOTAL,243818.01,227311.26,6333.85,2691.83,...,32098,224904.58,951,5969.18,848,5034.27,521,3607.32,520,4302.66


In [4]:
df['unicap'] = df['unicap'].astype('object')
df['renglon'] = df['renglon'].astype('object')
df['fecha_corte'] = pd.to_datetime(df['fecha_corte'])

# 3. Correlaciones variables numéricas

In [5]:
cols_num = df.select_dtypes(exclude=['object', 'datetime'])
df_corr = cols_num.corr()
fig = px.imshow(df_corr[(df_corr > 0.7) | (df_corr < -0.7)],  text_auto=True)
fig.update_layout(height=1000)
fig.show()

## ¿Por qué el saldo a la fecha y el saldo vigente presentan una correlación tan alta?

El saldo a la fecha y el saldo vigente presentan una alta correlación. El saldo vigente representa el saldo luego de los abonos de los clientes a cada una de las cuentas en cada mes. Por tanto, se decide descartar el saldo vigente ya que saldo a la fecha y los abonos nos brindan la misma información.

In [6]:
df.drop('vigente', axis=1, inplace=True)

## ¿Qué hacer con las altas correlaciones en los rangos de vencimiento?

Las demás columnas, aunque tengan una alta correlación, se decide conservarlas porque representan características diferentes de las cuentas. Mes a mes cambia el tiempo que lleva una cuenta vencida. Algunos rangos de vencimiento se traslapan, esto puede ser porque cada entidad maneja diferentes estrategias de cobranza dependiendo del tipo de producto y por tanto estipula diferentes rangos para establecer el tiempo que lleva una cuenta vencida.

Una entidad, dependiendo del riesgo, para un producto implementa una estrategia para cuentas de 1 a 3 meses vencidas, mientras que para otro producto aplica una estrategia si la cuenta tiene entre 1 y 2 meses vencida y otra estrategia si la cuenta tiene entre 2 y 3 meses vencida.

In [7]:
cols_mora = df.iloc[:,8:21].columns
df.groupby(['tipo_entidad', 'desc_unicap'])[cols_mora].sum().iloc[1,:].to_frame()

,COMPAÑIAS DE FINANCIAMIENTO
,CARTERA COMERCIAL CORPORATIVO
v2_3,0.00
v1_3,525695.79
v3_4,0.00
v+4,0.00
v3_6,257962.56
v+6,0.00
v1_4,0.00
v4_6,0.00
v6_12,142877.74


Por ejemplo, una entidad de tipo 1, para carteras comerciales corporativas clasifica las cuentas cuentas vencidad de 1 a 3 meses, de 3 a 6 meses, de 6 a 12 meses y más de 12 meses.

In [8]:
df.groupby(['tipo_entidad', 'desc_unicap'])[cols_mora].sum().iloc[10,:].to_frame()

,COMPAÑIAS DE FINANCIAMIENTO
,CONSUMO BAJO MONTO
v2_3,7451.78
v1_3,0.00
v3_4,0.00
v+4,0.00
v3_6,15553.00
v+6,45.50
v1_4,0.00
v4_6,0.00
v6_12,0.00


Mientras que la misma entidad, para cuentas de consumos de bajo monto, las clasifica de 1 a 2 meses, de 2 a 3 meses, de 3 a 6 meses y más de 6 meses.

Decidimos trabajar con todas las variables y observar los resultados obtenidos con reducción de dimensionalidad.

## ¿Qué hacer con las correlaciones altas en algunos riesgos?

In [9]:
df['year_month'] = pd.to_datetime(df.fecha_corte.apply(lambda x: str(x.year) + '-' + str(x.month) + '-' + '01'))
base = df.groupby(['year_month'])[['saldo_a_la_fecha','riesgo_a_s','riesgo_b_s','riesgo_c_s','riesgo_d_s','riesgo_e_s']].mean()
fig = px.line(base)
fig.show()

In [10]:
base2 = df.groupby(['year_month'])[['riesgo_a_c','riesgo_b_c','riesgo_c_c','riesgo_d_c','riesgo_e_c']].mean()
fig = px.line(base2)
fig.show()

La gran mayoría de los clientes tienen obligaciones catalogadas como de riesgo a, que es la categoría de menos riesgo para las entidades. Dado que las columnas saldo por riesgo representan cada una el saldo total de cuentas que hay de dichos riesgos y la gran mayoría son de riesgos a, hay una correlación casi de 100% entre el saldo de cuentas de riesgo a y el saldo total a la fecha, se puede observar que varían casi que igual en el tiempo. Dicho lo anterior, se decide conservar el saldo total a la fecha y descartar las columnas de saldo por riesgos. Las columnas de número de clientes por riesgos también representa muy bien la frecuencia e importancia de cada riesgos.

In [11]:
df.drop(['riesgo_a_s', 'riesgo_b_s', 'riesgo_c_s', 'riesgo_d_s', 'riesgo_e_s'], axis=1, inplace=True)

## ¿Qué hacer con las correlaciones altas de los riesgos con el número de clientes con más de 1 mes en mora?

In [12]:
base3 = df.groupby(['year_month'])[['clientes_mora+30d','riesgo_a_c','riesgo_b_c','riesgo_c_c','riesgo_d_c','riesgo_e_c']].mean()
fig = px.scatter_matrix(base3)
fig.update_layout(height=800)
fig.show()

Dado que esta variable presenta una alta correlación con otras 4 variables, se decide descartarla.

In [13]:
df.drop(['clientes_mora+30d'], axis=1, inplace=True)

## Observar correlación

In [14]:
cols_num = df.select_dtypes(exclude=['object', 'datetime'])
df_corr = cols_num.corr()
fig = px.imshow(df_corr,  text_auto=True)
fig.update_layout(height=1000)
fig.show()

# 4. Variables categóricas

In [15]:
df_cat = df.select_dtypes(include='object')
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80900 entries, 0 to 80899
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tipo_entidad  80900 non-null  object
 1   unicap        80900 non-null  object
 2   desc_unicap   80900 non-null  object
 3   renglon       80900 non-null  object
 4   desc_renglon  80900 non-null  object
dtypes: object(5)
memory usage: 3.1+ MB


In [16]:
from scipy.stats import chi2_contingency
tabla_contingencia1 = pd.crosstab(df['tipo_entidad'], df['desc_unicap'])
tabla_contingencia2 = pd.crosstab(df['tipo_entidad'], df['desc_renglon'])
tabla_contingencia3 = pd.crosstab(df['desc_renglon'], df['desc_unicap'])
chi21, p1, _, _ = chi2_contingency(tabla_contingencia1)
chi22, p2, _, _ = chi2_contingency(tabla_contingencia2)
chi23, p3, _, _ = chi2_contingency(tabla_contingencia3)

print('tipo_entidad vs desc_unicap:', p1)
print('tipo_entidad vs desc_renglon:', p2)
print('desc_renglon vs desc_unicap:', p3)

tipo_entidad vs desc_unicap: 0.0
tipo_entidad vs desc_renglon: 0.0
desc_renglon vs desc_unicap: 0.0


Las tres variables presentan correlación, ya que es p-valor de las pruebas chi cuadrado fue menor a 0.05.

El tipo de entidad son los clusters a los que queremos llegar con el análisis, por tanto, esta no se tiene en cuenta para descartarla por su correlación. Decidimos eliminar unicap, ya que renglon son categorías dentro de cada categoría de unicap, por tanto, contiene información más detallada sobre los tipos de productos.

In [17]:
df.drop(['unicap','desc_unicap'], axis=1, inplace=True)

# 5. Exportar data sin correlación

In [18]:
df.to_csv('datasets/data_no_corr.csv', index=False)